In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import os
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from memoization import cached
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Download Data yf_download

In [3]:
@cached
def yf_download(symbol, start, end):
    df = yf.download(symbol, start, end)[["Adj Close"]]
    return df
AAPL = yf_download("AAPL", "2016-01-01", "2018-01-01")
GOOG = yf_download("GOOG", "2016-01-01", "2018-01-01")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [20]:
# Put ETF_5years.csv in your directory
etfs = pd.read_csv("ETF.csv").Symbol.tolist()
if os.path.exists("ETF_5years.csv"):
    data = pd.read_csv("ETF_5years.csv", index_col=0)
else:
    data = yf_download(etfs[0], "2017-01-01", "2022-01-01")
    data.columns = [etfs[0]]
    for etf in etfs[1:]:
        data_etf = yf_download(etf, "2017-01-01", "2022-01-01")
        data_etf.columns = [etf]
        if len(data_etf) < 1200:
            continue
        data = data.merge(data_etf, left_index=True, right_index=True, how="left")
    data.to_csv("ETF_5years.csv")

## Test a pair trading strategy

### Get the Hedge Ratio from PCA

In [21]:
from sklearn.decomposition import PCA

def get_hedgeRatio(df):
    pca = PCA().fit(df)
    hedgeRatio = pca.components_[0][1] / pca.components_[0][0]
    return hedgeRatio

## An simple strategy
Long when difference is larger than 2 * $\sigma$, close when it is smaller than $\sigma$

Short when difference is larger than 2 * $\sigma$, close when it is smaller than $\sigma$

In [112]:
def strategy_profit(df):
    df["Hedged"] = df.iloc[:,0] * get_hedgeRatio(df)
    df["dif"] = df["Hedged"] - df.iloc[:,1]
    mu = df['dif'][:252*3].mean()
    sigma = df['dif'][:252*3].std()

    money = 0
    long = False
    short = False
    lookback = 252 * 3
    hists = []
    mu = df['dif'][:lookback].mean()
    u1, u2, d1, d2 = 1, 2, 1, 2

    for index, price in df["dif"][lookback:].iteritems():
        if price < mu - d2 * sigma and not long:
            long = True
            money -= price 
        if price > mu - d1 * sigma and long:
            long = False 
            money += price 
        if price > mu + u2 * sigma and not short:  
            short = True 
            money += price 
        if price < mu + u1 * sigma and short:
            short = False 
            money -= price 
        hists.append([index, price, long, short, money])
        
    if short:
        money -= price 
    if long:
        money += price 

    hists.append([index, price, False, False, money])

    print("Final money", round(money, 2))
    df_hist = pd.DataFrame(hists, columns=["date", "price_dif", "long", "short", "money"])

    def clean(x):
        return x if not np.isnan(x) else 0
    long_cost = clean(df_hist[df_hist["long"] == True]["price_dif"].mean())
    long_period = clean(df_hist[df_hist["long"] == True]["price_dif"].count())

    short_cost = clean(df_hist[df_hist["short"] == True]["price_dif"].mean())
    short_period = clean(df_hist[df_hist["short"] == True]["price_dif"].count())
    
    interest_cost = (abs(long_cost * long_period) + abs(short_cost * short_period)) / 252 * 0.05
    print("Cost of interest", interest_cost)
    return df_hist

## For example, we compare the first and second ETF

In [114]:
df = data.iloc[:, [0,1]]
strategy_profit(df)

Final money 12.98
Cost of interest 0.0055376989065770155


,date,price_dif,long,short,money
0,2020-01-06,0.836909,False,False,0.000000
1,2020-01-07,0.811324,False,False,0.000000
2,2020-01-08,0.890932,False,False,0.000000
3,2020-01-09,0.906140,False,False,0.000000
4,2020-01-10,0.870543,False,False,0.000000
...,...,...,...,...,...
499,2021-12-28,0.867239,False,False,12.983208
500,2021-12-29,0.981127,False,False,12.983208
501,2021-12-30,1.002747,False,False,12.983208
502,2021-12-31,0.985119,False,False,12.983208


In [115]:
n = 10
for i in range(n):
    for j in range(i+1, n):
        strategy_profit(data.iloc[:,[i,j]])

Final money 12.98
Cost of interest 0.0055376989065770155


,date,price_dif,long,short,money
0,2020-01-06,0.836909,False,False,0.000000
1,2020-01-07,0.811324,False,False,0.000000
2,2020-01-08,0.890932,False,False,0.000000
3,2020-01-09,0.906140,False,False,0.000000
4,2020-01-10,0.870543,False,False,0.000000
...,...,...,...,...,...
499,2021-12-28,0.867239,False,False,12.983208
500,2021-12-29,0.981127,False,False,12.983208
501,2021-12-30,1.002747,False,False,12.983208
502,2021-12-31,0.985119,False,False,12.983208


Final money 1.87
Cost of interest 0.235505745571093


,date,price_dif,long,short,money
0,2020-01-06,4.044245,False,True,4.044245
1,2020-01-07,3.962583,False,True,4.044245
2,2020-01-08,4.046295,False,True,4.044245
3,2020-01-09,4.159224,False,True,4.044245
4,2020-01-10,4.149640,False,True,4.044245
...,...,...,...,...,...
499,2021-12-28,6.299331,False,True,8.188714
500,2021-12-29,6.477547,False,True,8.188714
501,2021-12-30,6.178964,False,True,8.188714
502,2021-12-31,6.322982,False,True,8.188714


Final money 10.22
Cost of interest 0.007293662574591376


...

,date,price_dif,long,short,money
0,2020-01-06,-70.410509,False,False,0.000000
1,2020-01-07,-70.441462,False,False,0.000000
2,2020-01-08,-70.227669,False,False,0.000000
3,2020-01-09,-70.168103,False,False,0.000000
4,2020-01-10,-70.477648,False,False,0.000000
...,...,...,...,...,...
499,2021-12-28,-64.788580,False,False,3.349558
500,2021-12-29,-64.348506,False,False,3.349558
501,2021-12-30,-64.661840,False,False,3.349558
502,2021-12-31,-64.601780,False,False,3.349558


Final money 4.14
Cost of interest 0.13118116356268827


,date,price_dif,long,short,money
0,2020-01-06,-7.768290,False,False,0.000000
1,2020-01-07,-7.883038,False,False,0.000000
2,2020-01-08,-7.992315,False,False,0.000000
3,2020-01-09,-8.051641,False,False,0.000000
4,2020-01-10,-8.377821,False,False,0.000000
...,...,...,...,...,...
499,2021-12-28,-4.074606,False,False,3.599194
500,2021-12-29,-3.772537,False,True,-0.173343
501,2021-12-30,-4.460849,False,True,-0.173343
502,2021-12-31,-4.312435,False,True,-0.173343


Final money 12.34
Cost of interest 1.607169930439908


,date,price_dif,long,short,money
0,2020-01-06,30.789834,False,False,0.000000
1,2020-01-07,30.730949,False,False,0.000000
2,2020-01-08,30.455763,False,False,0.000000
3,2020-01-09,30.314378,False,False,0.000000
4,2020-01-10,30.228006,False,False,0.000000
...,...,...,...,...,...
499,2021-12-28,27.993853,False,False,12.341245
500,2021-12-29,27.974139,False,False,12.341245
501,2021-12-30,27.513047,False,False,12.341245
502,2021-12-31,27.616020,False,False,12.341245


In [55]:
# num_shares_google = 15
# google_price = df.iloc[0, 0]
# google_size = num_shares_google * google_price
# r = 0.0009

# aapl_price = df.iloc[0, 1]
# num_shares_aapl = np.around(-google_size/aapl_price) 
# aapl_size = aapl_price * num_shares_aapl
# initial_leverage = google_price

# google_size, aapl_size, initial_leverage

# delta_t = 1 / (2*252)
# equity = [100000]

# leverage = [initial_leverage]
# profits = [0]
# previous_googl = df.iloc[0, 0]
# previous_aapl = df.iloc[0, 1]
# one_lege_size = 25000